
# **Empirical Expriments of Anchor-MoE**


# Public Dataset

10 UCI benchmark regression datasets


In [ ]:
!pip install ngboost
!pip install ucimlrepo

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 13.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4119 sha256=15a25619e34ae2d1c50e9c3586dace334dfe581f32863180afb9658467f7f8be
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


# **Model Definition**

In [ ]:
#!/usr/bin/env python3
# anchor_moe_coupled_anchor_feature.py
# Anchor-MoE: GBDT(mean anchor) + MoE(variance [+tiny mean delta])
# - Coupling: feed mu_anchor_z as an extra input feature to MoE (router & experts see it)
# - Fix1: top-k smoothing only within top-k; non-topk strictly zero
# - Fix2: calibration leakage removed (hold-out CAL from 90%)
# NLL on z-score(y); RMSE on original y (with linear post-cal on CAL)
# 2025-08-15

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # force CPU if CUDA env is flaky

import math, gc
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ============== Globals ==============
LOG2PI  = math.log(2*math.pi)
DEVICE  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float32)

# ============== Utils ==============
def rmse_score(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# ============== Data loader (clean, last column is y) ==============
def load_dataset(name: str):
    def _clean_numeric(df: pd.DataFrame) -> pd.DataFrame:
        df = df.apply(pd.to_numeric, errors="coerce")
        df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
        return df

    name = name.lower().strip()

    if name == "yacht":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
            header=None, sep=r"\s+", engine="python", comment="#", skip_blank_lines=True
        )
        if df.shape[1] == 1:
            df = df[0].astype(str).str.strip().str.split(r"\s+", expand=True)
        df = _clean_numeric(df); assert df.shape[1] >= 7
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[Yacht] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "housing":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
            header=None, sep=r"\s+", engine="python", comment="#", skip_blank_lines=True
        )
        if df.shape[1] == 1:
            df = df[0].astype(str).str.strip().str.split(r"\s+", expand=True)
        df = _clean_numeric(df); assert df.shape[1] >= 14
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[Housing] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "concrete":
        df = pd.read_excel(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
        )
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[Concrete] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "wine":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
            delimiter=";"
        )
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[WineRed] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "power":
       local = "power.xlsx"  # 本地文件（UCI 原文件名通常为 Folds5x2_pp.xlsx）
       if not os.path.exists(local):
          raise FileNotFoundError(
            "[power] 需要本地文件 'power.xlsx'（UCI CCPP 的 Folds5x2_pp.xlsx）。")
       df = pd.read_excel(local)
       df = _clean_numeric(df)
    if name == "energy":
        df = pd.read_excel(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
        ).iloc[:, :-1]
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64)  # last column (Y2) as target
        y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[Energy] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "protein":
       local = "protein.csv"  # 改成你的实际路径
       df = pd.read_csv(local, sep=None, engine="python")  # 自动检测逗号/制表符/分号等
       df = _clean_numeric(df)
       y = df.iloc[:, 0].to_numpy(np.float64)
       X = df.iloc[:, 1:].to_numpy(np.float64)
       y_min, y_max = float(np.nanmin(y)), float(np.nanmax(y))
       print(f"[Protein(local)] X.shape={X.shape} y.shape={y.shape} | y∈[{y_min:.3f},{y_max:.3f}]")
       return X, y

    if name == "naval":
       local = "naval.txt"  # 按需改成你的实际路径
       df = pd.read_csv(local, sep=r"\s+", header=None, engine="python")
       df = df.iloc[:, :-1]
       df = _clean_numeric(df)
       X = df.iloc[:, :-1].to_numpy(np.float64)
       y = df.iloc[:, -1].to_numpy(np.float64)
       print(f"[Naval(local)] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.6f},{y.max():.6f}]")
       return X, y

    if name == "kin8nm":
        local = "kin8nm.csv"
        if not os.path.exists(local): raise FileNotFoundError("[kin8nm] require local 'kin8nm.csv'.")
        df = pd.read_csv(local); df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[Kin8nm] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "msd":
        local = "YearPredictionMSD.txt"
        if not os.path.exists(local): raise FileNotFoundError("[msd] require local 'YearPredictionMSD.txt'.")
        df = pd.read_csv(local, header=None); df = df.iloc[:, ::-1]
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        print(f"[MSD] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    raise ValueError("Unknown dataset. options: 'housing','concrete','wine','kin8nm','naval','power','energy','protein','yacht','msd'")

# ============== y's z-score (for NLL) ==============
def zscore_fit(y_train):
    my = float(y_train.mean()); sy = float(y_train.std() + 1e-8)
    return my, sy

# ============== Top-k helpers ==============
def _topk_mask(w, k=4):
    _, topi = torch.topk(w, k, dim=-1)
    mask = torch.zeros_like(w).scatter_(-1, topi, 1.0)
    w2 = w * mask
    return w2 / (w2.sum(dim=-1, keepdim=True) + 1e-12)

def _topk_mask_smooth(w, k=2, eps=0.05):
    # Smooth only within top-k; non-topk strictly zero
    _, topi = torch.topk(w, k, dim=-1)
    mask = torch.zeros_like(w).scatter_(-1, topi, 1.0)
    w_top = w * mask
    w_top = w_top / (w_top.sum(dim=-1, keepdim=True) + 1e-12)
    return (1.0 - eps) * w_top + (eps / k) * mask

# ============== Anchor-MoE modules ==============
class Projection(nn.Module):
    def __init__(self, d, D):
        super().__init__()
        self.w = nn.Linear(d, D, bias=True)
        nn.init.xavier_uniform_(self.w.weight); nn.init.zeros_(self.w.bias)
    def forward(self, x): return self.w(x)

class Window(nn.Module):
    def __init__(self, K, D, min_log_s=-2.5, max_log_s=1.0):
        super().__init__()
        self.c         = nn.Parameter(torch.randn(K, D))
        self.log_s     = nn.Parameter(torch.zeros(K, D))
        self.min_log_s = min_log_s; self.max_log_s = max_log_s
    def forward(self, z):
        log_s = torch.clamp(self.log_s, min=self.min_log_s, max=self.max_log_s)
        diff2 = ((z[:, None] - self.c)**2) / (2 * torch.exp(log_s)**2)
        return torch.exp(-diff2.sum(dim=-1)) + 1e-12  # [B,K]

class Router(nn.Module):
    def __init__(self, D, K):
        super().__init__()
        self.q   = nn.Linear(D, 64)
        self.k   = nn.Parameter(torch.randn(K, 64))
        self.tau = 3.0
    def forward(self, z):
        logits = (self.q(z) @ self.k.T) / math.sqrt(64)
        return F.softmax(logits / self.tau, dim=-1)

class ExpertMDN(nn.Module):
    def __init__(self, d, h, nc, sigma_min=5e-2, sigma_max=1.0, learn_mean=True):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, h), nn.ReLU(),
            nn.Linear(h, h), nn.ReLU()
        )
        self.logits = nn.Linear(h, nc)
        self.learn_mean = learn_mean
        if learn_mean:
            self.means  = nn.Linear(h, nc)
        self.log_sc = nn.Linear(h, nc)
        self.sigma_min = float(sigma_min)
        self.sigma_max = float(sigma_max)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
    def forward(self, x):
        h  = self.net(x)
        pi = F.softmax(self.logits(h), dim=-1)
        mu = self.means(h) if self.learn_mean else None
        sg = torch.exp(self.log_sc(h)).clamp(self.sigma_min, self.sigma_max)
        return pi, mu, sg

class BLRMoE(nn.Module):
    """
    mean_mode: 'anchor' | 'anchor+delta' | 'free'
    """
    def __init__(self, d, D, K, hid, nc,
                 mean_mode='anchor+delta', delta_l2=1e-3,
                 w_ent_warm=+1e-3, w_ent_cool=-2e-4, l2_win=1e-4, lb_coef=1e-3,
                 sigma_min=5e-2, sigma_max=1.0,
                 topk=2, smooth_eps=0.05):
        super().__init__()
        assert mean_mode in ['anchor','anchor+delta','free']
        self.mean_mode = mean_mode
        self.delta_l2  = float(delta_l2)
        self.proj   = Projection(d, D)
        self.win    = Window(K, D)
        self.router = Router(D, K)
        learn_mean = (mean_mode != 'anchor')
        self.exps   = nn.ModuleList([ExpertMDN(d, hid, nc, sigma_min, sigma_max, learn_mean=learn_mean) for _ in range(K)])
        self.w_ent_warm = w_ent_warm
        self.w_ent_cool = w_ent_cool
        self.l2_win     = l2_win
        self.lb_coef    = lb_coef
        self.topk       = topk
        self.smooth_eps = smooth_eps
        self.K = K; self.nc = nc

    def _mixture_params(self, X, train=True):
        z = self.proj(X)
        w = self.win(z) * self.router(z)
        w = w / (w.sum(dim=-1, keepdim=True) + 1e-12)
        w = _topk_mask_smooth(w, k=self.topk, eps=self.smooth_eps) if train else _topk_mask(w, k=self.topk)

        # Only compute params for active experts (top-k)
        B, K, C = X.size(0), self.K, self.nc
        Pi = torch.full((B, K, C), 1.0/C, device=X.device)
        Mu = torch.zeros(B, K, C, device=X.device)
        Sg = torch.ones(B, K, C, device=X.device)

        _, topi = torch.topk(w, self.topk, dim=-1)
        uniq = torch.unique(topi)
        for j in uniq.tolist():
            pi_j, mu_j, sg_j = self.exps[j](X)  # [B,C]
            mask = (topi == j).any(dim=1).float().unsqueeze(-1)  # [B,1]
            Pi[:, j, :] = pi_j * mask + Pi[:, j, :]*(1 - mask)
            if mu_j is not None:
                Mu[:, j, :] = mu_j * mask + Mu[:, j, :]*(1 - mask)
            Sg[:, j, :] = sg_j * mask + Sg[:, j, :]*(1 - mask)

        return w, Pi, Mu, Sg, z

    def nll(self, X, y_z, mu_anchor_z=None, epoch=1, warmup_epochs=150):
        train_flag = self.training
        w, Pi, Mu, Sg, z = self._mixture_params(X, train=train_flag)

        if self.mean_mode == 'anchor':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None]
            delta_pen = 0.0
        elif self.mean_mode == 'anchor+delta':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None] + Mu
            delta_pen = (Mu**2).mean() * self.delta_l2
        else:
            Mu_eff = Mu
            delta_pen = 0.0

        yv = y_z[:, None, None]
        logp = -0.5 * ((yv - Mu_eff)/Sg)**2 - torch.log(Sg) - 0.5*LOG2PI  # [B,K,C]

        w3 = w[:, :, None]
        logw = torch.where(w3 > 0, torch.log(w3 + 1e-12), torch.full_like(w3, -1e9))
        logpi= torch.log(Pi + 1e-12)

        logmix = torch.logsumexp(logw + logpi + logp, dim=(1,2))
        nll = -logmix.mean()

        if train_flag:
            w_ent = self.w_ent_warm if epoch <= warmup_epochs else self.w_ent_cool
            p = self.router(z)  # [B,K]
            ent = (p * torch.log(p + 1e-12)).sum(dim=1).mean()
            l2w = (self.win.log_s**2).mean()
            rho = p.mean(dim=0)
            lb_loss = ((rho - 1.0/p.size(1))**2).sum()

            nll = (nll
                   + w_ent * ent
                   + self.l2_win * l2w
                   + self.lb_coef * lb_loss
                   + delta_pen)
        return nll

    @torch.no_grad()
    def predict_mean_var(self, X, mu_anchor_z=None):
        w, Pi, Mu, Sg, _ = self._mixture_params(X, train=False)
        if self.mean_mode == 'anchor':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None]
        elif self.mean_mode == 'anchor+delta':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None] + Mu
        else:
            Mu_eff = Mu

        mu_e  = (Pi * Mu_eff).sum(dim=2)              # [B,K]
        m2_e  = (Pi * (Sg**2 + Mu_eff**2)).sum(dim=2) # [B,K]
        mu_z  = (w * mu_e).sum(dim=1)                 # [B]
        second= (w * m2_e).sum(dim=1)                 # [B]
        var_z = torch.clamp(second - mu_z**2, min=1e-9)
        return mu_z, var_z

# ============== Training (coupled anchor feature + no-leak calibration) ==============
def train_one_split_with_anchor(
    X_all, y_all, X_te, y_te,
    standardize_x=True,
    D=2, K=8, HID=128, NC=3,
    LR=1e-3, EPOCHS=400,
    MEAN_MODE='anchor+delta',
    DELTA_L2=1e-3,
    SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
    TOPK=2, SMOOTH_EPS=0.05
):
    # ---- 90% (X_all,y_all) -> TV + CAL (CAL held-out for calibration only) ----
    X_tv, X_cal, y_tv, y_cal = train_test_split(X_all, y_all, test_size=0.125, random_state=1)
    # ---- TV -> train/val for early stopping ----
    X_tr, X_va, y_tr, y_va   = train_test_split(X_tv,  y_tv,  test_size=0.2,   random_state=1)

    # ========= 1) GBDT: select best_it on (X_tr,X_va), retrain on TV =========
    gbdt_full_tr = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05,
                                             max_depth=3, random_state=1, subsample=1.0)
    gbdt_full_tr.fit(X_tr, y_tr)
    val_rmse = [rmse_score(y_va, p) for p in gbdt_full_tr.staged_predict(X_va)]
    best_it  = int(np.argmin(val_rmse)) + 1

    gbdt_sub   = GradientBoostingRegressor(n_estimators=best_it, learning_rate=0.05,
                                           max_depth=3, random_state=1, subsample=1.0).fit(X_tr, y_tr)
    gbdt_final = GradientBoostingRegressor(n_estimators=best_it, learning_rate=0.05,
                                           max_depth=3, random_state=1, subsample=1.0).fit(X_tv, y_tv)

    # ========= 2) Phase-1 MoE on (X_tr,X_va) with Phase-1 anchors =========
    my_tr, sy_tr = zscore_fit(y_tr)   # stats for Phase-1
    my_tv, sy_tv = zscore_fit(y_tv)   # stats for Phase-2 (TV)

    # anchors (z) for TR/VA produced by gbdt_sub and normalized by (my_tr, sy_tr)
    mu_tr_anchor = (gbdt_sub.predict(X_tr) - my_tr) / sy_tr
    mu_va_anchor = (gbdt_sub.predict(X_va) - my_tr) / sy_tr

    # ---- Coupling: append mu_anchor_z as an extra feature ----
    X_tr_aug = np.column_stack([X_tr, mu_tr_anchor])
    X_va_aug = np.column_stack([X_va, mu_va_anchor])

    # standardize features (including the anchor feature)
    if standardize_x:
        mx_tr = X_tr_aug.mean(0, keepdims=True); sx_tr = X_tr_aug.std(0, keepdims=True) + 1e-8
    else:
        mx_tr = np.zeros((1, X_tr_aug.shape[1])); sx_tr = np.ones((1, X_tr_aug.shape[1]))

    # helper to tensorize
    def to_tensor(x, yz=None, mx=None, sx=None):
        Xt = torch.tensor(((x - mx)/sx).astype(np.float32), device=DEVICE)
        if yz is None: return Xt
        Yt = torch.tensor(yz.astype(np.float32), device=DEVICE)
        return Xt, Yt

    y_tr_z = (y_tr - my_tr) / sy_tr
    y_va_z = (y_va - my_tr) / sy_tr

    model = BLRMoE(
        d=X_tr_aug.shape[1], D=D, K=K, hid=HID, nc=NC,
        mean_mode=MEAN_MODE, delta_l2=DELTA_L2,
        sigma_min=SIGMA_MIN, sigma_max=SIGMA_MAX,
        topk=TOPK, smooth_eps=SMOOTH_EPS
    ).to(DEVICE)
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=3e-4)

    Xtr_t, ytr_t = to_tensor(X_tr_aug, y_tr_z, mx_tr, sx_tr)
    Xva_t, yva_t = to_tensor(X_va_aug, y_va_z, mx_tr, sx_tr)
    mu_tr_t = torch.tensor(mu_tr_anchor.astype(np.float32), device=DEVICE)
    mu_va_t = torch.tensor(mu_va_anchor.astype(np.float32), device=DEVICE)

    best_ep, best_vnll, best_state = 0, +1e9, None
    for ep in range(1, EPOCHS+1):
        model.train()
        opt.zero_grad()
        loss = model.nll(Xtr_t, ytr_t, mu_anchor_z=mu_tr_t, epoch=ep, warmup_epochs=150)
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 2.0); opt.step()
        model.router.tau = max(model.router.tau * 0.995, 1.0)

        model.eval()
        with torch.no_grad():
            vnll = float(model.nll(Xva_t, yva_t, mu_anchor_z=mu_va_t, epoch=ep, warmup_epochs=150).cpu().item())
        if vnll < best_vnll:
            best_vnll = vnll; best_ep = ep
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    # ========= 3) Phase-2 retrain on TV only; make anchors from TV model =========
    mu_tv_anchor = (gbdt_final.predict(X_tv) - my_tv) / sy_tv
    mu_te_anchor = (gbdt_final.predict(X_te) - my_tv) / sy_tv
    mu_cal_anchor= (gbdt_final.predict(X_cal)- my_tv) / sy_tv

    # augment TV/TE/CAL with anchor feature
    X_tv_aug  = np.column_stack([X_tv,  mu_tv_anchor])
    X_te_aug  = np.column_stack([X_te,  mu_te_anchor])
    X_cal_aug = np.column_stack([X_cal, mu_cal_anchor])

    # standardization for Phase-2 on TV Aug
    if standardize_x:
        mx_tv = X_tv_aug.mean(0, keepdims=True); sx_tv = X_tv_aug.std(0, keepdims=True) + 1e-8
    else:
        mx_tv = np.zeros((1, X_tv_aug.shape[1])); sx_tv = np.ones((1, X_tv_aug.shape[1]))

    y_tv_z = (y_tv - my_tv) / sy_tv
    y_te_z = (y_te - my_tv) / sy_tv

    model.load_state_dict(best_state)
    model.train()
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=3e-4)

    Xtv_t, ytv_t = to_tensor(X_tv_aug, y_tv_z, mx_tv, sx_tv)
    Xte_t        = to_tensor(X_te_aug, None,   mx_tv, sx_tv)
    Xcal_t       = to_tensor(X_cal_aug, None,  mx_tv, sx_tv)
    mu_tv_t  = torch.tensor(mu_tv_anchor.astype(np.float32),  device=DEVICE)
    mu_te_t  = torch.tensor(mu_te_anchor.astype(np.float32),  device=DEVICE)
    mu_cal_t = torch.tensor(mu_cal_anchor.astype(np.float32), device=DEVICE)

    for ep in range(1, best_ep+1):
        opt.zero_grad()
        loss = model.nll(Xtv_t, ytv_t, mu_anchor_z=mu_tv_t, epoch=ep, warmup_epochs=150)
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 2.0); opt.step()
        model.router.tau = max(model.router.tau * 0.995, 1.0)

    # ========= 4) Calibration on CAL only; evaluate on test =========
    model.eval()
    with torch.no_grad():
        # Test NLL (z) — using TV stats and anchors
        yte_t = torch.tensor(y_te_z.astype(np.float32), device=DEVICE)
        test_nll = float(model.nll(Xte_t, yte_t, mu_anchor_z=mu_te_t).cpu().item())

        # Means in original units
        mu_z_cal, _ = model.predict_mean_var(Xcal_t, mu_anchor_z=mu_cal_t)
        mu_cal_orig = mu_z_cal.cpu().numpy().astype(np.float64) * sy_tv + my_tv

        mu_z_te, _  = model.predict_mean_var(Xte_t,  mu_anchor_z=mu_te_t)
        mu_te_orig  = mu_z_te.cpu().numpy().astype(np.float64) * sy_tv + my_tv

    # linear post-calibration on CAL
    A = np.vstack([mu_cal_orig, np.ones_like(mu_cal_orig)]).T
    ab, *_ = np.linalg.lstsq(A, y_cal.astype(np.float64), rcond=None)
    a, b = float(ab[0]), float(ab[1])

    mu_te_cal  = a * mu_te_orig + b
    rmse = rmse_score(y_te.astype(np.float64), mu_te_cal)

    return rmse, test_nll, best_it, best_ep

# ============== Main (20 random splits) ==============
def main():
    DATASET = "housing"  # change to 'housing','concrete','energy', ...
    X, y = load_dataset(DATASET)
    print(f"== Dataset={DATASET} X.shape={X.shape}")

    SEED = 1
    np.random.seed(SEED); torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    STANDARDIZE_X = True
    D, K, HID, NC = 2, 8, 128, 3
    LR, EPOCHS    = 1e-3, 400
    MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
    SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
    TOPK, SMOOTH_EPS = 2, 0.05

    n = X.shape[0]
    splits = []
    for _ in range(20):
        perm = np.random.choice(np.arange(n), n, replace=False)
        splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

    rmses, nlls = [], []
    for itr, (tr_idx, te_idx) in enumerate(splits, 1):
        rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
            X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
            standardize_x=STANDARDIZE_X,
            D=D, K=K, HID=HID, NC=NC,
            LR=LR, EPOCHS=EPOCHS,
            MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
            SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
            TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
        )
        print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
              f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
        rmses.append(rmse); nlls.append(nll)
        gc.collect();
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    rmses = np.array(rmses, dtype=np.float64)
    nlls  = np.array(nlls,  dtype=np.float64)
    se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
    print("\n== Anchor-MoE (coupled anchor feature, no-leak, topk-fixed) ==")
    print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
    print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

#if __name__ == "__main__":
#    main()

# **Boston Dataset**

In [ ]:
# === Run on Housing (UCI Boston) ===

X, y = load_dataset("housing")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Housing ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Housing] X.shape=(506, 13) y.shape=(506,) | y∈[5.000,50.000]
[01/20] GBDT best_iter=289  MoE best_ep=10  TestRMSE(orig)=2.5337  TestNLL(z)=0.3435
[02/20] GBDT best_iter=305  MoE best_ep=10  TestRMSE(orig)=2.5833  TestNLL(z)=0.2538
[03/20] GBDT best_iter=545  MoE best_ep=11  TestRMSE(orig)=2.1390  TestNLL(z)=0.1778
[04/20] GBDT best_iter=455  MoE best_ep=13  TestRMSE(orig)=3.8278  TestNLL(z)=0.2141
[05/20] GBDT best_iter=253  MoE best_ep=14  TestRMSE(orig)=3.7482  TestNLL(z)=1.1760
[06/20] GBDT best_iter=298  MoE best_ep=10  TestRMSE(orig)=2.5234  TestNLL(z)=0.2582
[07/20] GBDT best_iter=421  MoE best_ep=10  TestRMSE(orig)=2.1363  TestNLL(z)=0.1458
[08/20] GBDT best_iter=234  MoE best_ep=11  TestRMSE(orig)=2.6484  TestNLL(z)=0.3162
[09/20] GBDT best_iter=552  MoE best_ep=9  TestRMSE(orig)=3.5282  TestNLL(z)=0.8218
[10/20] GBDT best_iter=416  MoE best_ep=11  TestRMSE(orig)=4.8476  TestNLL(z)=1.1085
[11/20] GBDT best_iter=388  MoE best_ep=17  TestRMSE(orig)=3.5469  TestNLL(z)=2.2304
[12/

# **Concrete**

In [ ]:
# === Run on Housing (UCI concrete) ===

X, y = load_dataset("concrete")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on concrete ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Concrete] X.shape=(1030, 8) y.shape=(1030,) | y∈[2.332,82.599]
[01/20] GBDT best_iter=1981  MoE best_ep=12  TestRMSE(orig)=4.9270  TestNLL(z)=0.2130
[02/20] GBDT best_iter=1823  MoE best_ep=17  TestRMSE(orig)=3.3315  TestNLL(z)=-0.0048
[03/20] GBDT best_iter=1204  MoE best_ep=17  TestRMSE(orig)=2.9753  TestNLL(z)=-0.3177
[04/20] GBDT best_iter=1989  MoE best_ep=15  TestRMSE(orig)=4.3188  TestNLL(z)=0.0002
[05/20] GBDT best_iter=2000  MoE best_ep=14  TestRMSE(orig)=4.9339  TestNLL(z)=0.4477
[06/20] GBDT best_iter=1999  MoE best_ep=14  TestRMSE(orig)=3.8787  TestNLL(z)=0.1072
[07/20] GBDT best_iter=2000  MoE best_ep=15  TestRMSE(orig)=5.4025  TestNLL(z)=0.7132
[08/20] GBDT best_iter=1949  MoE best_ep=16  TestRMSE(orig)=4.7943  TestNLL(z)=0.1700
[09/20] GBDT best_iter=1523  MoE best_ep=13  TestRMSE(orig)=4.1883  TestNLL(z)=-0.0573
[10/20] GBDT best_iter=779  MoE best_ep=18  TestRMSE(orig)=5.1184  TestNLL(z)=0.5029
[11/20] GBDT best_iter=943  MoE best_ep=15  TestRMSE(orig)=4.0216  TestNLL

# **Energy**

In [ ]:
# === Run on Housing (UCI Energy) ===

X, y = load_dataset("energy")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Energy ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Energy] X.shape=(768, 8) y.shape=(768,) | y∈[6.010,43.100]
[01/20] GBDT best_iter=1986  MoE best_ep=172  TestRMSE(orig)=0.3912  TestNLL(z)=-1.7609
[02/20] GBDT best_iter=1826  MoE best_ep=326  TestRMSE(orig)=0.5551  TestNLL(z)=-1.4699
[03/20] GBDT best_iter=1982  MoE best_ep=231  TestRMSE(orig)=0.3986  TestNLL(z)=-1.7945
[04/20] GBDT best_iter=2000  MoE best_ep=391  TestRMSE(orig)=0.4544  TestNLL(z)=-1.6483
[05/20] GBDT best_iter=1828  MoE best_ep=186  TestRMSE(orig)=0.4850  TestNLL(z)=-1.6297
[06/20] GBDT best_iter=1962  MoE best_ep=257  TestRMSE(orig)=0.3891  TestNLL(z)=-1.7654
[07/20] GBDT best_iter=1949  MoE best_ep=40  TestRMSE(orig)=0.7941  TestNLL(z)=-1.5950
[08/20] GBDT best_iter=1996  MoE best_ep=130  TestRMSE(orig)=0.3606  TestNLL(z)=-1.7804
[09/20] GBDT best_iter=1999  MoE best_ep=221  TestRMSE(orig)=0.4991  TestNLL(z)=-1.5604
[10/20] GBDT best_iter=1706  MoE best_ep=376  TestRMSE(orig)=0.3565  TestNLL(z)=-1.7761
[11/20] GBDT best_iter=1652  MoE best_ep=85  TestRMSE(orig)=0

# **Kin8nm**

In [ ]:
# === RFF-KRR Mixture (GP-inspired MoE) on Kin8nm ===
# - 20 x random 90/10 splits, NLL in z-score(y), RMSE in original y
# - Each expert is a kernel ridge regressor (RBF) via Random Fourier Features
# - Gating is RBF distance from KMeans centers (no y leakage)
# - K=1 -> exactly one KRR (≈ single RBF-GP in function form)
# ---------------------------------------------------
import numpy as np, math, gc
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# ---------- 0) dataset ----------
try:
    X, y = load_dataset("kin8nm")   # 你 notebook 里已有
except NameError:
    import pandas as pd, os
    assert os.path.exists("kin8nm.csv"), "缺少本地 kin8nm.csv"
    df = pd.read_csv("kin8nm.csv")
    X = df.iloc[:, :-1].to_numpy(np.float64)
    y = df.iloc[:,  -1].to_numpy(np.float64)
    print(f"[Kin8nm] (fallback) X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
print(f"== Dataset=Kin8nm X.shape={X.shape} y.shape={y.shape} ==")

# 可选：子采样（更快对比）
SUBSAMPLE = None   # 设为 None 则用全部
if SUBSAMPLE is not None and X.shape[0] > SUBSAMPLE:
    idx = np.random.RandomState(1).choice(X.shape[0], SUBSAMPLE, replace=False)
    X, y = X[idx], y[idx]
    print(f"[Kin8nm] subsampled: X.shape={X.shape}, y.shape={y.shape}")

# ---------- 1) protocol ----------
SEED = 1
rng = np.random.RandomState(SEED)
n = X.shape[0]
splits = []
for _ in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

# ---------- 2) helpers ----------
def rmse(a, b): return float(np.sqrt(np.mean((a - b)**2)))
def zfit(y_tr):
    my = float(y_tr.mean()); sy = float(y_tr.std() + 1e-8)
    return my, sy

# RFF feature map for RBF kernel: phi(x) = sqrt(2/D) * cos(W x + b)
class RFF:
    def __init__(self, d_in, n_feat=256, lengthscale=1.0, seed=0):
        rs = np.random.RandomState(seed)
        self.n_feat = int(n_feat)
        self.lengthscale = float(lengthscale)
        # sample omega ~ N(0, I), scaled by 1/lengthscale
        self.W = rs.normal(size=(d_in, self.n_feat)) / max(self.lengthscale, 1e-6)
        self.b = rs.uniform(0.0, 2*np.pi, size=(self.n_feat,))
        self.scale = np.sqrt(2.0 / self.n_feat)
    def features(self, X):  # X: [N,d]
        Z = X @ self.W + self.b
        return self.scale * np.cos(Z)  # [N,m]

# Weighted ridge: (Phi^T W Phi + λI)^{-1} Phi^T W y
def weighted_ridge(Phi, y, w=None, lam=1e-3):
    # Phi: [N,m], y: [N], w: [N] or None
    if w is None:
        A = Phi.T @ Phi + lam * np.eye(Phi.shape[1])
        b = Phi.T @ y
    else:
        W = w[:, None]
        A = Phi.T @ (W * Phi) + lam * np.eye(Phi.shape[1])
        b = Phi.T @ (w * y)
    w_lin = np.linalg.solve(A, b)
    return w_lin, A  # 返回 A 以便近似方差时用（可选）

def mixture_loglik_gaussians(yz, mus, sig2, mix_w):
    """
    yz: [N] 真实 z-score 标签
    mus: [N,K] 各专家均值
    sig2:[K]   各专家方差（常数；也可做成 [N,K]）
    mix_w:[N,K] 混合权重（和为1）
    返回：每样本 log p(yz)
    """
    LOG2PI = math.log(2*math.pi)
    # 广播为 [N,K]
    var = sig2[None, :]
    ll = -0.5*((yz[:, None] - mus)**2)/var - 0.5*np.log(var) - 0.5*LOG2PI
    # log-sum-exp over K
    a = np.log(mix_w + 1e-12) + ll
    a_max = a.max(axis=1, keepdims=True)
    logp = a_max[:,0] + np.log(np.exp(a - a_max).sum(axis=1) + 1e-12)
    return logp

def run_one_split(X_tr_full, y_tr_full, X_te, y_te,
                  K=3, n_feat=256, lam=1e-3, topk=None,
                  use_anchor=False,  # 若 True：拟合 z 中的残差（anchor+delta）
                  random_state=0):
    # --- 标准化 X（仅用训练） ---
    scaler = StandardScaler().fit(X_tr_full)
    Xtr = scaler.transform(X_tr_full)
    Xte = scaler.transform(X_te)

    # --- z-score(y)（仅用训练） ---
    my, sy = zfit(y_tr_full)
    ytr_z = (y_tr_full - my) / sy
    yte_z = (y_te      - my) / sy

    # --- 可选 anchor（保持与之前口径一致，默认先关） ---
    if use_anchor:
        from sklearn.ensemble import GradientBoostingRegressor
        gbdt = GradientBoostingRegressor(n_estimators=1500, learning_rate=0.05,
                                         max_depth=3, random_state=random_state)
        gbdt.fit(X_tr_full, y_tr_full)
        mu_tr_anchor_z = (gbdt.predict(X_tr_full) - my) / sy
        mu_te_anchor_z = (gbdt.predict(X_te)      - my) / sy
        target_tr = ytr_z - mu_tr_anchor_z
    else:
        mu_tr_anchor_z = np.zeros_like(ytr_z)
        mu_te_anchor_z = np.zeros_like(yte_z)
        target_tr = ytr_z

    # --- gating：KMeans + RBF 距离软分配（不看 y，无泄漏） ---
    km = KMeans(n_clusters=K, n_init=10, random_state=random_state).fit(Xtr)
    C = km.cluster_centers_                  # [K,d]
    # 用各簇内均方距离作为带宽（简易自适应）
    labels = km.labels_
    bw2 = []
    for k in range(K):
        pts = Xtr[labels == k]
        if pts.shape[0] < 2:
            bw2.append(1.0)
        else:
            bw2.append(np.mean(np.sum((pts - C[k])**2, axis=1)) + 1e-6)
    bw2 = np.asarray(bw2)  # [K]

    def soft_assign(Xin):
        # RBF 距离 → softmax
        D2 = np.sum((Xin[:, None, :] - C[None, :, :])**2, axis=2)  # [N,K]
        logits = - D2 / (2.0 * bw2[None, :])
        P = np.exp(logits - logits.max(axis=1, keepdims=True))
        P = P / (P.sum(axis=1, keepdims=True) + 1e-12)
        if topk is not None and topk < K:
            # 稀疏化（仅 top-k 保留），再归一化
            idx = np.argsort(-P, axis=1)[:, :topk]
            mask = np.zeros_like(P); mask[np.arange(P.shape[0])[:,None], idx] = 1
            P = P * mask
            P = P / (P.sum(axis=1, keepdims=True) + 1e-12)
        return P  # [N,K]

    W_tr = soft_assign(Xtr)   # 训练软权重（仅 X 驱动）

    # --- 为每个专家拟合 RFF-KRR（带样本权重） ---
    N, d = Xtr.shape
    experts = []
    mus_tr = np.zeros((N, K))
    for k in range(K):
        rff = RFF(d_in=d, n_feat=n_feat, lengthscale=np.sqrt(bw2[k]), seed=random_state+100+k)
        Phi = rff.features(Xtr)                        # [N,m]
        w_k, A_k = weighted_ridge(Phi, target_tr, w=W_tr[:, k], lam=lam)
        mu_k_tr  = Phi @ w_k                           # [N]
        # 专家噪声：加权残差方差（防止过小）
        resid    = target_tr - mu_k_tr
        wsum     = W_tr[:, k].sum() + 1e-12
        sig2_k   = max(float((W_tr[:, k] * (resid**2)).sum() / wsum), 1e-8)
        experts.append((rff, w_k, A_k, sig2_k))
        mus_tr[:, k] = mu_k_tr

    # --- 测试：组合为混合高斯 ---
    W_te = soft_assign(Xte)                      # [Nt,K]
    Nt = Xte.shape[0]
    mus_te = np.zeros((Nt, K))
    sig2   = np.zeros((K,), dtype=np.float64)
    for k, (rff, w_k, A_k, sig2_k) in enumerate(experts):
        Phi_te = rff.features(Xte)
        mus_te[:, k] = Phi_te @ w_k
        sig2[k] = sig2_k
        # 如需更接近 GP，可把“模型不确定度”也加上：
        # var_model = np.sum(Phi_te @ np.linalg.inv(A_k) * Phi_te, axis=1)
        # 但这会很花时间；先用常数噪声近似，往往已足够稳定做对比。

    # 恢复到 y_z 的均值：anchor 若启用则加回 anchor
    mus_te_full = mus_te + mu_te_anchor_z[:, None]

    # NLL（z）
    logp = mixture_loglik_gaussians(yte_z, mus_te_full, sig2, W_te)
    nll = -float(np.mean(logp))

    # RMSE（orig）
    mu_pred_z = np.sum(W_te * mus_te_full, axis=1)       # 混合均值
    mu_pred   = mu_pred_z * sy + my
    rmse_val  = rmse(mu_pred, y_te)

    return rmse_val, nll

# ---------- 3) run ----------
K = 20              # 专家数；K=1 退化为单 KRR（≈单 GP）
N_FEAT = 4096      # RFF 维度（越大越接近 RBF-GP，越慢）
LAMBDA = 1e-3     # 岭系数
TOPK = K        # 可设为 K 或更小做稀疏 gating
USE_ANCHOR = None  # True 则做 anchor+delta（残差回归）；先关，保持与 GP 口径一致

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_te, y_te = X[te_idx], y[te_idx]

    r, n = run_one_split(
        X_tr, y_tr, X_te, y_te,
        K=K, n_feat=N_FEAT, lam=LAMBDA, topk=TOPK,
        use_anchor=USE_ANCHOR, random_state=SEED + i
    )
    print(f"[{i:02d}/20] K={K}  RFF={N_FEAT}  lambda={LAMBDA:.1e} | TestRMSE={r:.4f}  TestNLL(z)={n:.4f}")
    rmses.append(r); nlls.append(n); gc.collect()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== RFF-KRR Mixture (GP-inspired MoE) on Kin8nm ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Kin8nm] (fallback) X.shape=(8192, 8) y.shape=(8192,) | y∈[0.040,1.459]
== Dataset=Kin8nm X.shape=(8192, 8) y.shape=(8192,) ==
[01/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0744  TestNLL(z)=0.1546
[02/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0735  TestNLL(z)=0.1298
[03/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0724  TestNLL(z)=0.1181
[04/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0722  TestNLL(z)=0.1047
[05/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0715  TestNLL(z)=0.1026
[06/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0775  TestNLL(z)=0.2042
[07/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0737  TestNLL(z)=0.1404
[08/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0743  TestNLL(z)=0.1366
[09/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0681  TestNLL(z)=0.0575
[10/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0776  TestNLL(z)=0.1769
[11/20] K=20  RFF=4096  lambda=1.0e-03 | TestRMSE=0.0692  TestNLL(z)=0.0741
[12/20] K=20  RFF=4096  lambda=1.0e-0

# **Naval**

In [ ]:
# === Run on Housing (UCI Boston) ===

X, y = load_dataset("naval")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Housing ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Naval(local)] X.shape=(11934, 16) y.shape=(11934,) | y∈[0.950000,1.000000]
[01/20] GBDT best_iter=1999  MoE best_ep=215  TestRMSE(orig)=0.0011  TestNLL(z)=-1.1336
[02/20] GBDT best_iter=2000  MoE best_ep=322  TestRMSE(orig)=0.0009  TestNLL(z)=-1.3619
[03/20] GBDT best_iter=2000  MoE best_ep=280  TestRMSE(orig)=0.0009  TestNLL(z)=-1.4272
[04/20] GBDT best_iter=2000  MoE best_ep=110  TestRMSE(orig)=0.0010  TestNLL(z)=-1.2368
[05/20] GBDT best_iter=2000  MoE best_ep=130  TestRMSE(orig)=0.0010  TestNLL(z)=-1.2014
[06/20] GBDT best_iter=2000  MoE best_ep=372  TestRMSE(orig)=0.0010  TestNLL(z)=-1.3299
[07/20] GBDT best_iter=2000  MoE best_ep=399  TestRMSE(orig)=0.0010  TestNLL(z)=-1.2364
[08/20] GBDT best_iter=1998  MoE best_ep=235  TestRMSE(orig)=0.0010  TestNLL(z)=-1.2702
[09/20] GBDT best_iter=1999  MoE best_ep=145  TestRMSE(orig)=0.0011  TestNLL(z)=-1.2576
[10/20] GBDT best_iter=2000  MoE best_ep=369  TestRMSE(orig)=0.0010  TestNLL(z)=-1.2053
[11/20] GBDT best_iter=2000  MoE best_ep=159

# Power Plant

In [ ]:
# === Run on Housing (UCI Boston) ===

X, y = load_dataset("power")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Housing ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")


[Power] X.shape=(9568, 4) y.shape=(9568,) | y∈[420.260,495.760]
[01/20] GBDT best_iter=1999  MoE best_ep=20  TestRMSE(orig)=3.2548  TestNLL(z)=-0.1435
[02/20] GBDT best_iter=1998  MoE best_ep=20  TestRMSE(orig)=3.3316  TestNLL(z)=0.0056
[03/20] GBDT best_iter=1997  MoE best_ep=21  TestRMSE(orig)=2.9826  TestNLL(z)=-0.1756
[04/20] GBDT best_iter=1994  MoE best_ep=42  TestRMSE(orig)=3.0454  TestNLL(z)=-0.2052
[05/20] GBDT best_iter=1998  MoE best_ep=22  TestRMSE(orig)=3.6840  TestNLL(z)=-0.1179
[06/20] GBDT best_iter=1992  MoE best_ep=26  TestRMSE(orig)=3.4787  TestNLL(z)=-0.0480
[07/20] GBDT best_iter=1999  MoE best_ep=18  TestRMSE(orig)=3.1511  TestNLL(z)=-0.1883
[08/20] GBDT best_iter=1999  MoE best_ep=27  TestRMSE(orig)=3.0624  TestNLL(z)=-0.0496
[09/20] GBDT best_iter=1991  MoE best_ep=20  TestRMSE(orig)=3.4551  TestNLL(z)=-0.1624
[10/20] GBDT best_iter=1996  MoE best_ep=21  TestRMSE(orig)=3.3850  TestNLL(z)=-0.1650
[11/20] GBDT best_iter=1999  MoE best_ep=24  TestRMSE(orig)=3.0285 

# **Protein**

In [ ]:
# === Run on Protein (UCI CASP) with SUBSAMPLE=10000 ===
import numpy as np, gc, torch

X, y = load_dataset("protein")  # 本地 protein.csv：第一列为 y，其余为 X

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------- Subsample to 10k BEFORE creating splits --------
N_SUB = 10000
n_full = X.shape[0]
if n_full > N_SUB:
    rng_sub = np.random.RandomState(SEED)
    idx_sub = rng_sub.choice(n_full, N_SUB, replace=False)
    X, y = X[idx_sub], y[idx_sub]
    print(f"[Protein] subsampled to {N_SUB} from {n_full}")
else:
    print(f"[Protein] dataset size {n_full} ≤ {N_SUB}, use full data")

# （可选）为了让外层切分可复现，重置一次种子
np.random.seed(SEED)

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Protein (subsampled to 10k) ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Protein(local)] X.shape=(45730, 9) y.shape=(45730,) | y∈[0.000,20.999]
[Protein] subsampled to 10000 from 45730
[01/20] GBDT best_iter=1820  MoE best_ep=68  TestRMSE(orig)=4.2760  TestNLL(z)=0.9240
[02/20] GBDT best_iter=1986  MoE best_ep=6  TestRMSE(orig)=4.4256  TestNLL(z)=1.1784
[03/20] GBDT best_iter=1685  MoE best_ep=4  TestRMSE(orig)=4.3811  TestNLL(z)=1.0640
[04/20] GBDT best_iter=1721  MoE best_ep=57  TestRMSE(orig)=4.4167  TestNLL(z)=1.0832
[05/20] GBDT best_iter=1998  MoE best_ep=5  TestRMSE(orig)=4.4133  TestNLL(z)=1.0971
[06/20] GBDT best_iter=1495  MoE best_ep=5  TestRMSE(orig)=4.3702  TestNLL(z)=1.0654
[07/20] GBDT best_iter=1178  MoE best_ep=114  TestRMSE(orig)=4.5190  TestNLL(z)=0.8933
[08/20] GBDT best_iter=1995  MoE best_ep=2  TestRMSE(orig)=4.3996  TestNLL(z)=1.1519
[09/20] GBDT best_iter=1704  MoE best_ep=7  TestRMSE(orig)=4.3023  TestNLL(z)=1.1320
[10/20] GBDT best_iter=1965  MoE best_ep=5  TestRMSE(orig)=4.4172  TestNLL(z)=1.0711
[11/20] GBDT best_iter=1667  MoE 

In [ ]:
# === NGBoost baseline on Protein (subsampled to 10k), NLL in z(y), RMSE in original y ===
import numpy as np, math, gc, os, sys, subprocess

# 1) ensure ngboost
try:
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    from ngboost.scores import MLE
except Exception as e:
    print("[setup] Installing ngboost ...")
    subprocess.check_call([sys._getframe(0).f_globals.get("sys").executable, "-m", "pip", "install", "ngboost"])
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    from ngboost.scores import MLE

# 2) dataset
try:
    X, y = load_dataset("protein")   # 你已经按“第一列 y，其余 X”的本地版本改好
except NameError:
    import pandas as pd
    assert os.path.exists("protein.csv"), "缺少本地 protein.csv（第一列 y，其余为 X）"
    df = pd.read_csv("protein.csv")
    y = df.iloc[:, 0].to_numpy(np.float64)
    X = df.iloc[:, 1:].to_numpy(np.float64)
print(f"== Dataset=Protein(local) X.shape={X.shape} y.shape={y.shape} ==")

# 3) subsample to 10k BEFORE splits
SEED = 1
RS = np.random.RandomState(SEED)
N_SUB = 10000
n_full = X.shape[0]
if n_full > N_SUB:
    idx_sub = RS.choice(n_full, N_SUB, replace=False)
    X, y = X[idx_sub], y[idx_sub]
    print(f"[Protein] subsampled to {N_SUB} from {n_full}")
else:
    print(f"[Protein] dataset size {n_full} ≤ {N_SUB}, use full data")

# 4) 20x 90/10 outer splits (same protocol)
RS = np.random.RandomState(SEED)  # reset for reproducibility
n = X.shape[0]
splits = []
for _ in range(20):
    perm = RS.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

def rmse(a, b): return float(np.sqrt(np.mean((a - b)**2)))

def zfit(y_tr):
    my = float(y_tr.mean()); sy = float(y_tr.std() + 1e-8)
    return my, sy

LOG2PI = math.log(2*math.pi)

# 5) run NGBoost on each split
rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_te, y_te = X[te_idx], y[te_idx]

    # z-score(y) by train stats
    my, sy = zfit(y_tr)
    y_te_z = (y_te - my) / sy

    # NGBoost model (Normal likelihood)
    # 为了与之前 GBDT 的复杂度接近，这里给一个较强配置；如需更快可下调 n_estimators
    ngb = NGBRegressor(
        Dist=Normal,
        Score=MLE,
        n_estimators=2000,        # 你也可以设 3000（更强但更慢）
        learning_rate=0.05,       # 与你 GBDT 保持同量级
        natural_gradient=True,
        random_state=SEED + i,
        verbose=False
    )
    ngb.fit(X_tr, y_tr)

    # predicted distribution on test (original space)
    dist = ngb.pred_dist(X_te)     # Normal distribution object
    mu  = dist.loc                 # mean in original space, shape [Nt]
    sig = dist.scale               # std  in original space, shape [Nt]
    sig = np.maximum(sig, 1e-12)

    # map to z-space for NLL
    mu_z  = (mu - my) / sy
    sig_z = sig / sy
    sig2_z = np.maximum(sig_z**2, 1e-12)

    # NLL in z(y): mean over test set
    nll = 0.5*np.log(sig2_z) + 0.5*((y_te_z - mu_z)**2)/sig2_z + 0.5*LOG2PI
    nll = float(np.mean(nll))

    # RMSE in original space
    rmse_val = rmse(mu, y_te)

    print(f"[{i:02d}/20] TestRMSE={rmse_val:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse_val); nlls.append(nll)
    gc.collect()

rmses = np.array(rmses, np.float64); nlls = np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== NGBoost (Normal) on Protein (subsampled to 10k) ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Protein(local)] X.shape=(45730, 9) y.shape=(45730,) | y∈[0.000,20.999]
== Dataset=Protein(local) X.shape=(45730, 9) y.shape=(45730,) ==
[Protein] subsampled to 10000 from 45730
[01/20] TestRMSE=4.2750  TestNLL(z)=1.2519
[02/20] TestRMSE=4.5089  TestNLL(z)=1.2483
[03/20] TestRMSE=4.4202  TestNLL(z)=1.3973
[04/20] TestRMSE=4.3900  TestNLL(z)=1.1127
[05/20] TestRMSE=4.3676  TestNLL(z)=1.6895
[06/20] TestRMSE=4.4145  TestNLL(z)=1.3132
[07/20] TestRMSE=4.4981  TestNLL(z)=1.3205
[08/20] TestRMSE=4.4001  TestNLL(z)=1.1653
[09/20] TestRMSE=4.3782  TestNLL(z)=1.0529
[10/20] TestRMSE=4.4477  TestNLL(z)=1.1204
[11/20] TestRMSE=4.4191  TestNLL(z)=1.0913
[12/20] TestRMSE=4.4724  TestNLL(z)=1.3240
[13/20] TestRMSE=4.3791  TestNLL(z)=1.1726
[14/20] TestRMSE=4.4829  TestNLL(z)=0.9971
[15/20] TestRMSE=4.5001  TestNLL(z)=1.3051
[16/20] TestRMSE=4.4920  TestNLL(z)=1.1451
[17/20] TestRMSE=4.6766  TestNLL(z)=1.2180
[18/20] TestRMSE=4.5327  TestNLL(z)=1.3616
[19/20] TestRMSE=4.5361  TestNLL(z)=1.4836
[20/2

# **Wine Quality**


In [ ]:
# === Run on Housing (UCI Boston) ===

X, y = load_dataset("wine")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Housing ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")


[WineRed] X.shape=(1599, 11) y.shape=(1599,) | y∈[3.000,8.000]
[01/20] GBDT best_iter=390  MoE best_ep=3  TestRMSE(orig)=0.5995  TestNLL(z)=1.1511
[02/20] GBDT best_iter=229  MoE best_ep=3  TestRMSE(orig)=0.5963  TestNLL(z)=1.1316
[03/20] GBDT best_iter=96  MoE best_ep=2  TestRMSE(orig)=0.7258  TestNLL(z)=1.3395
[04/20] GBDT best_iter=95  MoE best_ep=2  TestRMSE(orig)=0.5908  TestNLL(z)=1.1001
[05/20] GBDT best_iter=437  MoE best_ep=3  TestRMSE(orig)=0.5632  TestNLL(z)=1.1395
[06/20] GBDT best_iter=337  MoE best_ep=3  TestRMSE(orig)=0.6185  TestNLL(z)=1.1745
[07/20] GBDT best_iter=1407  MoE best_ep=5  TestRMSE(orig)=0.6411  TestNLL(z)=1.3395
[08/20] GBDT best_iter=708  MoE best_ep=4  TestRMSE(orig)=0.6841  TestNLL(z)=1.3782
[09/20] GBDT best_iter=617  MoE best_ep=5  TestRMSE(orig)=0.5763  TestNLL(z)=1.1883
[10/20] GBDT best_iter=504  MoE best_ep=6  TestRMSE(orig)=0.6395  TestNLL(z)=1.3746
[11/20] GBDT best_iter=305  MoE best_ep=4  TestRMSE(orig)=0.6652  TestNLL(z)=1.2604
[12/20] GBDT b

In [ ]:
# === NGBoost baseline on Protein (subsampled to 10k), NLL in z(y), RMSE in original y ===
import numpy as np, math, gc, os, sys, subprocess

# 1) ensure ngboost
try:
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    from ngboost.scores import MLE
except Exception as e:
    print("[setup] Installing ngboost ...")
    subprocess.check_call([sys._getframe(0).f_globals.get("sys").executable, "-m", "pip", "install", "ngboost"])
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    from ngboost.scores import MLE

# 2) dataset
try:
    X, y = load_dataset("wine")   # 你已经按“第一列 y，其余 X”的本地版本改好
except NameError:
    import pandas as pd
    assert os.path.exists("protein.csv"), "缺少本地 protein.csv（第一列 y，其余为 X）"
    df = pd.read_csv("protein.csv")
    y = df.iloc[:, 0].to_numpy(np.float64)
    X = df.iloc[:, 1:].to_numpy(np.float64)
print(f"== Dataset=Protein(local) X.shape={X.shape} y.shape={y.shape} ==")

# 3) subsample to 10k BEFORE splits
SEED = 1
RS = np.random.RandomState(SEED)
N_SUB = 1599
n_full = X.shape[0]
if n_full > N_SUB:
    idx_sub = RS.choice(n_full, N_SUB, replace=False)
    X, y = X[idx_sub], y[idx_sub]
    print(f"[Protein] subsampled to {N_SUB} from {n_full}")
else:
    print(f"[Protein] dataset size {n_full} ≤ {N_SUB}, use full data")

# 4) 20x 90/10 outer splits (same protocol)
RS = np.random.RandomState(SEED)  # reset for reproducibility
n = X.shape[0]
splits = []
for _ in range(20):
    perm = RS.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

def rmse(a, b): return float(np.sqrt(np.mean((a - b)**2)))

def zfit(y_tr):
    my = float(y_tr.mean()); sy = float(y_tr.std() + 1e-8)
    return my, sy

LOG2PI = math.log(2*math.pi)

# 5) run NGBoost on each split
rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_te, y_te = X[te_idx], y[te_idx]

    # z-score(y) by train stats
    my, sy = zfit(y_tr)
    y_te_z = (y_te - my) / sy

    # NGBoost model (Normal likelihood)
    # 为了与之前 GBDT 的复杂度接近，这里给一个较强配置；如需更快可下调 n_estimators
    ngb = NGBRegressor(
        Dist=Normal,
        Score=MLE,
        n_estimators=2000,        # 你也可以设 3000（更强但更慢）
        learning_rate=0.05,       # 与你 GBDT 保持同量级
        natural_gradient=True,
        random_state=SEED + i,
        verbose=False
    )
    ngb.fit(X_tr, y_tr)

    # predicted distribution on test (original space)
    dist = ngb.pred_dist(X_te)     # Normal distribution object
    mu  = dist.loc                 # mean in original space, shape [Nt]
    sig = dist.scale               # std  in original space, shape [Nt]
    sig = np.maximum(sig, 1e-12)

    # map to z-space for NLL
    mu_z  = (mu - my) / sy
    sig_z = sig / sy
    sig2_z = np.maximum(sig_z**2, 1e-12)

    # NLL in z(y): mean over test set
    nll = 0.5*np.log(sig2_z) + 0.5*((y_te_z - mu_z)**2)/sig2_z + 0.5*LOG2PI
    nll = float(np.mean(nll))

    # RMSE in original space
    rmse_val = rmse(mu, y_te)

    print(f"[{i:02d}/20] TestRMSE={rmse_val:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse_val); nlls.append(nll)
    gc.collect()

rmses = np.array(rmses, np.float64); nlls = np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== NGBoost (Normal) on Protein (subsampled to 10k) ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[WineRed] X.shape=(1599, 11) y.shape=(1599,) | y∈[3.000,8.000]
== Dataset=Protein(local) X.shape=(1599, 11) y.shape=(1599,) ==
[Protein] dataset size 1599 ≤ 1599, use full data
[01/20] TestRMSE=0.5799  TestNLL(z)=2.7180
[02/20] TestRMSE=0.5681  TestNLL(z)=4.0791
[03/20] TestRMSE=0.7069  TestNLL(z)=6.5129
[04/20] TestRMSE=0.5870  TestNLL(z)=2.7675
[05/20] TestRMSE=0.5591  TestNLL(z)=2.2286
[06/20] TestRMSE=0.6033  TestNLL(z)=8.1670
[07/20] TestRMSE=0.6130  TestNLL(z)=2.0925
[08/20] TestRMSE=0.6593  TestNLL(z)=4.7736
[09/20] TestRMSE=0.5688  TestNLL(z)=2.7527
[10/20] TestRMSE=0.6168  TestNLL(z)=5.0409
[11/20] TestRMSE=0.6293  TestNLL(z)=10.5768
[12/20] TestRMSE=0.6307  TestNLL(z)=2.4453
[13/20] TestRMSE=0.6636  TestNLL(z)=6.7118
[14/20] TestRMSE=0.5923  TestNLL(z)=4.7667
[15/20] TestRMSE=0.5628  TestNLL(z)=3.7086
[16/20] TestRMSE=0.5304  TestNLL(z)=3.3801
[17/20] TestRMSE=0.6056  TestNLL(z)=2.4574
[18/20] TestRMSE=0.5718  TestNLL(z)=9.8242
[19/20] TestRMSE=0.5763  TestNLL(z)=5.0681
[20/2


# **Yacht**



In [ ]:
# === Run on Housing (UCI yacht) ===

X, y = load_dataset("yacht")

SEED = 1
np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 20 次 90/10 外层划分（与 NGBoost 协议一致）
n = X.shape[0]
splits = []
for i in range(20):
    perm = np.random.choice(np.arange(n), n, replace=False)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 超参与前面一致
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS = 1e-3, 400
MEAN_MODE, DELTA_L2 = 'anchor+delta', 3e-3
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for itr, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, m_it, moe_ep = train_one_split_with_anchor(
        X[tr_idx], y[tr_idx], X[te_idx], y[te_idx],
        standardize_x=STANDARDIZE_X,
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        MEAN_MODE=MEAN_MODE, DELTA_L2=DELTA_L2,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS
    )
    print(f"[{itr:02d}/20] GBDT best_iter={m_it}  MoE best_ep={moe_ep}  "
          f"TestRMSE(orig)={rmse:.4f}  TestNLL(z)={nll:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

rmses, nlls = np.array(rmses, np.float64), np.array(nlls, np.float64)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== Anchor-MoE (coupled anchor feature) on Housing ==")
print(f"RMSE (orig) = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  (z)    = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Yacht] X.shape=(308, 6) y.shape=(308,) | y∈[0.010,62.420]
[01/20] GBDT best_iter=1997  MoE best_ep=57  TestRMSE(orig)=0.7562  TestNLL(z)=-1.8476
[02/20] GBDT best_iter=1956  MoE best_ep=278  TestRMSE(orig)=0.6752  TestNLL(z)=-1.9226
[03/20] GBDT best_iter=1908  MoE best_ep=116  TestRMSE(orig)=0.3375  TestNLL(z)=-1.9768
[04/20] GBDT best_iter=2000  MoE best_ep=91  TestRMSE(orig)=0.5994  TestNLL(z)=-1.8201
[05/20] GBDT best_iter=1930  MoE best_ep=161  TestRMSE(orig)=0.6423  TestNLL(z)=-1.8032
[06/20] GBDT best_iter=1999  MoE best_ep=71  TestRMSE(orig)=0.2992  TestNLL(z)=-1.9629
[07/20] GBDT best_iter=1999  MoE best_ep=313  TestRMSE(orig)=0.2742  TestNLL(z)=-2.0070
[08/20] GBDT best_iter=1998  MoE best_ep=236  TestRMSE(orig)=0.4175  TestNLL(z)=-1.6997
[09/20] GBDT best_iter=1462  MoE best_ep=395  TestRMSE(orig)=0.6515  TestNLL(z)=-1.9014
[10/20] GBDT best_iter=1983  MoE best_ep=267  TestRMSE(orig)=0.3339  TestNLL(z)=-2.0049
[11/20] GBDT best_iter=1940  MoE best_ep=376  TestRMSE(orig)=0.4

# **LVMI**